In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# SETUP CELL — run this first
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, string, os

# Adjust this path to your chromedriver executable if needed
CHROMEDRIVER_PATH = r"C:\webdrivers\chromedriver.exe"

BASE_URL = "https://sourcebd-deployment.vercel.app"

def create_driver():
    options = Options()
    options.add_argument("--start-maximized")
    # Visible mode; remove detach so session is tied to notebook while running
    driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)
    driver.implicitly_wait(8)
    return driver

def rnd_email(prefix="rehan_test"):
    suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
    return f"{prefix}_{suffix}@example.com"

def save_screenshot(driver, name):
    folder = "selenium_screenshots"
    if not os.path.exists(folder):
        os.makedirs(folder)
    path = os.path.join(folder, name)
    driver.save_screenshot(path)
    print(f"📸 Screenshot saved: {path}")

# small helper wrappers
def wait_for_clickable(driver, locator, timeout=10):
    return WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))

def wait_for_visible(driver, locator, timeout=10):
    return WebDriverWait(driver, timeout).until(EC.visibility_of_element_located(locator))


In [17]:
# ✅ REGISTER PAGE TEST (for SourceBD live site)
from selenium.webdriver.common.by import By
import time

driver = create_driver()
driver.get("https://sourcebd-deployment.vercel.app/register")
driver.set_window_size(1280, 900)
time.sleep(2)

try:
    # Fill the registration form
    driver.find_element(By.ID, "firstName").send_keys("Rehan")
    driver.find_element(By.ID, "lastName").send_keys("Tester")
    driver.find_element(By.ID, "email").send_keys("rehan_test_2025@example.com")  # use unique email each time
    driver.find_element(By.ID, "phone").send_keys("01700000000")

    # Country (Bangladesh already selected, but this ensures it)
    country_dropdown = driver.find_element(By.ID, "country")
    for option in country_dropdown.find_elements(By.TAG_NAME, "option"):
        if option.text.strip() == "Bangladesh":
            option.click()
            break

    driver.find_element(By.ID, "password").send_keys("TestPassword123")
    driver.find_element(By.ID, "confirmPassword").send_keys("TestPassword123")

    # Click Create Account button
    submit_btn = driver.find_element(
        By.XPATH, "//button[@type='submit' and (contains(., 'Create Account') or contains(., 'Sign Up'))]"
    )
    driver.execute_script("arguments[0].click();", submit_btn)
    print("🖱️ Clicked Create Account button")

    # Wait for redirect or success message
    time.sleep(6)

    if "login" in driver.current_url.lower():
        print("✅ Registration successful (redirected to login)")
    elif "/" in driver.current_url.lower():
        print("✅ Registration successful (auto logged in and redirected to homepage)")
    else:
        print("⚠️ Registration submitted but not redirected, check manually")

except Exception as e:
    print("❌ Error during registration:", e)

time.sleep(3)
driver.quit()
print("✅ Browser closed after registration test")


🖱️ Clicked Create Account button
✅ Registration successful (auto logged in and redirected to homepage)
✅ Browser closed after registration test


In [25]:
# ✅ LOGIN PAGE TEST — SourceBD live site
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = create_driver()
driver.get("https://sourcebd-deployment.vercel.app/login")
driver.set_window_size(1280, 900)

try:
    # Wait until the email field is visible
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "email"))
    )

    # Fill in credentials (replace with valid account)
    driver.find_element(By.ID, "email").send_keys("rehan_test_2025@example.com")
    driver.find_element(By.ID, "password").send_keys("TestPassword123")

    # Wait for Sign In button and click it
    sign_in_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, "//button[@type='submit' and (normalize-space()='Sign In' or contains(., 'Sign In'))]")
        )
    )
    driver.execute_script("arguments[0].click();", sign_in_btn)
    print("🖱️ Clicked Sign In button")

    # Wait for redirect or success message
    time.sleep(4)

    current_url = driver.current_url
    print("🧭 Current URL:", current_url)

    # Check for success message or homepage redirect
    page_source = driver.page_source.lower()
    if "login successful" in page_source or "redirecting" in page_source:
        print("✅ Login success message detected")
    elif "login" not in current_url.lower():
        print("✅ Redirected to homepage — login successful")
    else:
        print("❌ Login may have failed (still on login page)")

except Exception as e:
    print("❌ Error during login test:", e)

time.sleep(3)
driver.quit()
print("✅ Browser closed after login test")


🖱️ Clicked Sign In button
🧭 Current URL: https://sourcebd-deployment.vercel.app/
✅ Redirected to homepage — login successful
✅ Browser closed after login test


In [26]:
# ✅ ADVANCED HOMEPAGE TEST (Header + Links + Search + Dropdown)
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = create_driver()
driver.get("https://sourcebd-deployment.vercel.app/")
driver.set_window_size(1280, 900)

try:
    # Wait until the header and search bar appear
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "header"))
    )
    print("✅ Homepage and header loaded")

    # --- Scroll test ---
    for _ in range(3):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    print("🧭 Scrolled through homepage")

    # --- Check header links (About, Contact, Help, Cart, etc.) ---
    header_links = driver.find_elements(By.CSS_SELECTOR, "header a")
    print(f"🔗 Found {len(header_links)} header links")

    for l in header_links:
        print(" →", l.text.strip())

    # --- Search bar test ---
    search_input = driver.find_element(By.CSS_SELECTOR, ".search-bar input[type='text']")
    search_input.clear()
    search_input.send_keys("Apple Watch")
    driver.find_element(By.CSS_SELECTOR, ".search-bar button").click()
    print("🔍 Performed search for 'dress'")
    time.sleep(3)

    # Wait for search results or page change
    if "search" in driver.current_url.lower():
        print("✅ Search redirected successfully:", driver.current_url)
    else:
        print("⚠️ Search did not navigate, check search handler")

    # --- Go back to homepage ---
    driver.get("https://sourcebd-deployment.vercel.app/")
    time.sleep(2)

    # --- If logged in: open profile dropdown ---
    try:
        profile_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".profile-btn"))
        )
        driver.execute_script("arguments[0].click();", profile_btn)
        print("👤 Opened profile dropdown")

        # Wait for dropdown to appear
        WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, ".dropdown-card"))
        )
        dropdown_items = driver.find_elements(By.CSS_SELECTOR, ".dropdown-item")
        for item in dropdown_items:
            print("   → Dropdown option:", item.text.strip())

        # Close dropdown by clicking outside
        driver.find_element(By.TAG_NAME, "body").click()
        print("📂 Closed profile dropdown")
    except Exception:
        print("⚠️ Profile dropdown not available (user likely not logged in)")

    # --- Check Cart link ---
    cart_link = driver.find_element(By.CSS_SELECTOR, ".cart-link")
    driver.execute_script("arguments[0].click();", cart_link)
    print("🛒 Clicked Cart link")
    time.sleep(2)
    print("✅ Cart page URL:", driver.current_url)

    # --- Return to homepage ---
    driver.get("https://sourcebd-deployment.vercel.app/")
    time.sleep(2)

    # --- Optional: About / Contact / Help links ---
    for label in ["About", "Contact", "Help"]:
        try:
            link = driver.find_element(By.XPATH, f"//a[contains(., '{label}')]")
            driver.execute_script("arguments[0].click();", link)
            print(f"📄 Opened {label} page")
            time.sleep(2)
            driver.back()
        except Exception:
            print(f"⚠️ {label} link not found or failed")

    print("✅ Homepage functional test completed successfully")

except Exception as e:
    print("❌ Homepage test failed:", e)

time.sleep(2)
driver.quit()
print("✅ Browser closed after homepage test")


✅ Homepage and header loaded
🧭 Scrolled through homepage
🔗 Found 6 header links
 → Source
Bd
 → Sign In
 → 
 → About Us
 → Contact
 → Help
🔍 Performed search for 'dress'
✅ Search redirected successfully: https://sourcebd-deployment.vercel.app/search?q=Apple%20Watch
⚠️ Profile dropdown not available (user likely not logged in)
🛒 Clicked Cart link
✅ Cart page URL: https://sourcebd-deployment.vercel.app/cart
📄 Opened About page
📄 Opened Contact page
📄 Opened Help page
✅ Homepage functional test completed successfully
✅ Browser closed after homepage test
